# Лабораторная работа №3
## Вариант 4
### 1. Вычислить средние рейтинги для фильма с ID 4 в зависимости от пола

#### 1.1. Считать данные

In [ ]:
import pandas as pd

movies = pd.read_table("ml-1m/movies.dat", sep="::",names=["MovieID", "Title", "Genres"], encoding="latin-1", engine="python")
ratings = pd.read_table("ml-1m/ratings.dat", sep="::", names=["UserID", "MovieID", "Rating", "Timestamp"], encoding="latin-1", engine="python")
users = pd.read_table("ml-1m/users.dat", sep="::", names=["UserID", "Gender", "Age", "Occupation", "Zip-code"], encoding="latin-1", engine="python")
display(movies)
display(ratings)
display(users)

#### 1.2. Объединить таблицы

In [ ]:
df = ratings.merge(users, on="UserID").merge(movies, on="MovieID")
display(df)

#### 1.3. Вычислить средний рейтинг по полу

In [ ]:
movie_id = 4 # № варианта
result1 = df[df["MovieID"] == movie_id].groupby("Gender")["Rating"].mean()
film = movies[movies["MovieID"] == movie_id]["Title"].values[0]
print(film)
display(result1)

### 2. Отобрать фильмы, получившие не менее 250 оценок, и для них определить средний рейтинг в зависимости от пола

#### 2.1. Сосчитать количество оценок на фильм

In [ ]:

rating_counts = (df.groupby("MovieID")).size()
popular_movies = rating_counts[rating_counts >= 250].index

#### 2.2. Отфильтровать популярные фильмы

In [ ]:
df_popular = df[df["MovieID"].isin(popular_movies)]

#### 2.3. Сгрупировать по MovieID и полу, посчитать средний рейтинг

In [ ]:
gender_ratings = df_popular.groupby(["MovieID", "Gender"])["Rating"].mean().unstack(fill_value=0)

#### 2.4. Привести к нужному формату

In [ ]:
result2 = gender_ratings.reset_index()
result2.columns = ["MovieID", "Rating_F", "Rating_M"]

#### 2.5. Добавить Title

In [ ]:
result2 = result2.merge(movies[["MovieID", "Title"]], on="MovieID")
result2 = result2[["MovieID", "Title", "Rating_F", "Rating_M"]]
display(result2)

### 3. Найти фильмы, оказавшиеся в топ 5 у зрителей-женщин и худшие 5 у зрителей-мужчин.   Найти   фильмы   (топ   5),   по   которым   мужчины   и   женщины   сильнее   всего разошлись в оценках

#### 3.1. Создать df с фильмами где есть оценки и от M и от F

In [ ]:
gender_ratings_full = df_popular.groupby(["MovieID", "Gender"])["Rating"].mean().unstack()
gender_ratings_full = gender_ratings_full.dropna()
display(gender_ratings_full)

#### 3.2. Объединить с movies и добавить заголовок

In [ ]:
gender_ratings_full = gender_ratings_full.merge(movies[["MovieID", "Title"]], on="MovieID")
display(gender_ratings_full)

#### 3.3. Определить топ 5 у женщин

In [ ]:
top_female = gender_ratings_full.nlargest(5, "F")[["MovieID", "Title", "F"]]
display(top_female)

#### 3.4. Определить 5 худших у мужчин

In [ ]:
worst_male = gender_ratings_full.nsmallest(5, "M")[["MovieID", "Title", "M"]]
display(worst_male)

#### 3.5. Определить топ 5 с наибольшим расхождением между мужчинами и женщинами

In [ ]:
gender_ratings_full["Diff"] = (gender_ratings_full["F"] - gender_ratings_full["M"]).abs()
top_diff = gender_ratings_full.nlargest(5, "Diff")[["MovieID", "Title", "Diff"]]
display(top_diff)

### 4. Определить топ 5 фильмов с наибольшим разногласием (независимо от пола)

In [ ]:
disagreement = df.groupby("MovieID")["Rating"].std().sort_values(ascending=False).head(5)
disagreement = disagreement.to_frame().merge(movies[["MovieID", "Title"]], on="MovieID")
display(disagreement)

### 5. Определить средний рейтинг каждого жанра для различных возрастов. Установите, фильмы какого жанра в каждой категории возрастов имеют наибольший и наименьший рейтинги.

### 5.1. Добавить читаемый возраст

In [ ]:
age_map = {
    1: "Under 18",
    18: "18-24",
    25: "25-34",
    35: "35-44",
    45: "45-49",
    50: "50-55",
    56: "56+"
}

df["AgeGroup"] = df["Age"].map(age_map)
display(df)

#### 5.2. Разделить по жанрам

In [ ]:
df_exploded = df.copy()
df_exploded["Genres"] = df_exploded["Genres"].str.split("|")
df_exploded = df_exploded.explode("Genres")
display(df_exploded)

#### 5.3. Сгрупировать по возрасту и жанру, сосчитать средний рейтинг

In [ ]:
genre_age_rating = df_exploded.groupby(["Genres", "AgeGroup"])["Rating"].mean().reset_index()
display(genre_age_rating)

#### 5.4. Определить жанр с наибольшим и наименьшим рейтингом по возрастным группам

In [ ]:
best_idx = genre_age_rating.groupby("AgeGroup")["Rating"].idxmax()
worst_idx = genre_age_rating.groupby("AgeGroup")["Rating"].idxmin()

# Получаем соответствующие строки
best_rows = genre_age_rating.loc[best_idx].set_index("AgeGroup")
worst_rows = genre_age_rating.loc[worst_idx].set_index("AgeGroup")

# Объединяем результаты
result3 = pd.DataFrame({
    "BestGenre": best_rows["Genres"],
    "BestRating": best_rows["Rating"],
    "WorstGenre": worst_rows["Genres"],
    "WorstRating": worst_rows["Rating"]
}).reset_index()
display(result3)